In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import random

In [2]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, activation_function='relu'):
        super(SimpleNN, self).__init__()

        if activation_function == 'relu':
            self.activation = nn.ReLU()
        elif activation_function == 'tanh':
            self.activation = nn.Tanh()

        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = self.activation(self.layer1(x))
        x = self.layer2(x)
        return x


In [3]:
def train_model(model, X_train_tensor, y_train_tensor, lr, batch_size, epochs=100):
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)

    model.train()
    for epoch in range(epochs):
        permutation = torch.randperm(X_train_tensor.size(0))

        for i in range(0, X_train_tensor.size(0), batch_size):
            indices = permutation[i:i+batch_size]
            batch_x, batch_y = X_train_tensor[indices], y_train_tensor[indices]

            optimizer.zero_grad()
            output = model(batch_x)
            loss = criterion(output, batch_y)
            loss.backward()
            optimizer.step()

    return model

In [5]:
def random_search_hyperparameters(X_train_tensor, y_train_tensor, num_trials=10):
    best_rmse = float('inf')
    best_params = {}

    learning_rates = [0.01, 0.1, 0.001]
    batch_sizes = [8, 16, 32]
    hidden_sizes = [4, 8, 16]
    activations = ['relu', 'tanh']

    for trial in range(num_trials):
        lr = random.choice(learning_rates)
        batch_size = random.choice(batch_sizes)
        hidden_size = random.choice(hidden_sizes)
        activation = random.choice(activations)

        print(f"Trial {trial+1}: Training with lr={lr}, batch_size={batch_size}, hidden_size={hidden_size}, activation={activation}")

        model = SimpleNN(input_size=X_train_tensor.shape[1], hidden_size=hidden_size, activation_function=activation)

        trained_model = train_model(model, X_train_tensor, y_train_tensor, lr, batch_size)

        with torch.no_grad():
            model.eval()
            output = trained_model(X_train_tensor)
            rmse = torch.sqrt(nn.MSELoss()(output, y_train_tensor))

        print(f"RMSE: {rmse.item():.4f}")

        if rmse.item() < best_rmse:
            best_rmse = rmse.item()
            best_params = {
                'learning_rate': lr,
                'batch_size': batch_size,
                'hidden_size': hidden_size,
                'activation_function': activation
            }

    return best_params, best_rmse

In [6]:
X, y = make_regression(n_samples=100, n_features=5, noise=0.1, random_state=42)

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [7]:
X_train_tensor = torch.tensor(X, dtype=torch.float32)
y_train_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)


In [8]:
best_params, best_rmse = random_search_hyperparameters(X_train_tensor, y_train_tensor, num_trials=10)

Trial 1: Training with lr=0.01, batch_size=8, hidden_size=16, activation=relu
RMSE: nan
Trial 2: Training with lr=0.1, batch_size=32, hidden_size=4, activation=tanh
RMSE: 72.7143
Trial 3: Training with lr=0.1, batch_size=32, hidden_size=16, activation=relu
RMSE: nan
Trial 4: Training with lr=0.1, batch_size=32, hidden_size=16, activation=relu
RMSE: nan
Trial 5: Training with lr=0.01, batch_size=16, hidden_size=16, activation=relu
RMSE: nan
Trial 6: Training with lr=0.01, batch_size=16, hidden_size=8, activation=relu
RMSE: nan
Trial 7: Training with lr=0.001, batch_size=16, hidden_size=8, activation=relu
RMSE: 0.2439
Trial 8: Training with lr=0.1, batch_size=8, hidden_size=16, activation=relu
RMSE: nan
Trial 9: Training with lr=0.01, batch_size=16, hidden_size=4, activation=relu
RMSE: 139.9331
Trial 10: Training with lr=0.1, batch_size=16, hidden_size=4, activation=tanh
RMSE: 74.4290


In [9]:
print("\nBest Hyperparameters:")
print(f"Learning Rate: {best_params['learning_rate']}")
print(f"Batch Size: {best_params['batch_size']}")
print(f"Hidden Size: {best_params['hidden_size']}")
print(f"Activation Function: {best_params['activation_function']}")
print(f"Best RMSE: {best_rmse:.4f}")


Best Hyperparameters:
Learning Rate: 0.001
Batch Size: 16
Hidden Size: 8
Activation Function: relu
Best RMSE: 0.2439
